In [3]:
import json
f = open("/local/home/stuff/datasets/crowdai/train/annotation-small.json")
  
# returns JSON object as 
# a dictionary
annotations = json.load(f)

In [4]:
print(annotations['annotations'][1])

{'id': 1792841, 'image_id': 210291, 'segmentation': [[49.0, -6.184466335694134e-15, 58.0, 10.999999999999995, 49.0, 22.999999999999993, 61.0, 37.99999999999999, 34.0, 74.0, 22.0, 59.99999999999999, 21.0, 60.99999999999999, 2.0, 37.99999999999999, 7.0, 31.999999999999993, 1.0, 24.99999999999999, 3.0, 20.99999999999999, 0.0, 15.999999999999991, 0.0, -9.184850993605149e-15, 49.0, -6.184466335694134e-15]], 'area': 3055.0, 'bbox': [-6.184466335694134e-15, 0.0, 74.0, 61.0], 'category_id': 100, 'iscrowd': 0}


In [5]:
dataset = [{'file_name': "/local/home/stuff/datasets/crowdai/train/images/68701.jpg",
            'image_id': 68701,
            'height': 300,
            'width': 300,
            'annotations': annotations['annotations'][0]},
            {'file_name': "/local/home/stuff/datasets/crowdai/train/images/210291.jpg",
             'image_id': 210291,
             'height': 300,
             'width': 300,
             'annotations': annotations['annotations'][1]}]

with open('dummy_annotations.json', 'w') as f:
    json.dump(dataset, f)

In [6]:
f = open("dummy_annotations.json")
  
# returns JSON object as 
# a dictionary
annotations = json.load(f)
print(annotations)

[{'file_name': '/local/home/stuff/datasets/crowdai/train/images/68701.jpg', 'image_id': 68701, 'height': 300, 'width': 300, 'annotations': {'id': 585133, 'image_id': 68701, 'segmentation': [[191.0, 95.0, 191.0, 28.999999999999996, 214.0, 28.999999999999996, 214.0, 52.99999999999999, 238.0, 52.99999999999999, 238.0, 100.0, 214.0, 100.0, 214.0, 95.0, 191.0, 95.0]], 'area': 2646.0, 'bbox': [28.999999999999996, 191.0, 209.0, 47.0], 'category_id': 100, 'iscrowd': 0}}, {'file_name': '/local/home/stuff/datasets/crowdai/train/images/210291.jpg', 'image_id': 210291, 'height': 300, 'width': 300, 'annotations': {'id': 1792841, 'image_id': 210291, 'segmentation': [[49.0, -6.184466335694134e-15, 58.0, 10.999999999999995, 49.0, 22.999999999999993, 61.0, 37.99999999999999, 34.0, 74.0, 22.0, 59.99999999999999, 21.0, 60.99999999999999, 2.0, 37.99999999999999, 7.0, 31.999999999999993, 1.0, 24.99999999999999, 3.0, 20.99999999999999, 0.0, 15.999999999999991, 0.0, -9.184850993605149e-15, 49.0, -6.184466335

In [7]:
import json
from detectron2.data import MetadataCatalog, DatasetCatalog


def load_data(t="train"):
    if t == "train":
        with open("dummy_annotations.json", 'r') as file:
            train = json.load(file)
        return train

for d in ["train"]:
    DatasetCatalog.register(d, lambda d=d: load_data(d))
    MetadataCatalog.get(d).set(thing_classes=["Building"])
metadata = MetadataCatalog.get("train")

/local/home/bsoyuer/miniconda3/envs/detectron/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:

from detectron2.config import get_cfg
from detectron2 import model_zoo


def custom_config(num_classes):
    cfg = get_cfg()
    
    # get configuration from model_zoo
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")

    # Model
    cfg.MODEL.MASK_ON = True
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    #cfg.MODEL.BACKBONE.NAME = "build_resnet_backbone"
    cfg.MODEL.RESNETS.DEPTH = 34
    cfg.MODEL.RESNETS.RES2_OUT_CHANNELS = 64
    #cfg.MODEL.RPN.IN_FEATURES = ['p3', 'p4', 'p5', 'p6', 'p7']
    
    
    # Solver
    cfg.SOLVER.BASE_LR = 0.0002
    cfg.SOLVER.MAX_ITER = 40000
    cfg.SOLVER.STEPS = (20, 10000, 20000)
    cfg.SOLVER.gamma = 0.5
    cfg.SOLVER.IMS_PER_BATCH = 4
    
    # Test
    cfg.TEST.DETECTIONS_PER_IMAGE = 20
    
    # INPUT
    #cfg.INPUT.MIN_SIZE_TRAIN = (300,)
    
    # DATASETS
    cfg.DATASETS.TEST = ('val',)
    cfg.DATASETS.TRAIN = ('train',)
    
    # DATASETS
    cfg.OUTPUT_DIR = "your local path"
    
    return cfg

In [20]:
print(custom_config(1))

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('val',)
  TRAIN: ('train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32], [64], [128], [256], [512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_fpn_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: ['res2', 'res3', 'res4', 'res

In [19]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

  

#for d in ["train", "val"]:
#    DatasetCatalog.register(d, lambda d=d: load_data(d))
#    MetadataCatalog.get(d).set(thing_classes=["Person", "Dog", "Cat"])
#metadata = MetadataCatalog.get("train")

cfg = custom_config(2)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.